In [ ]:
# default_exp main

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Loading

> Module to load the roster table.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
from pathlib import Path
import re
import pandas as pd
from typing import Any
from datetime import timedelta
from roster2ical.roster import Shift, ShiftProperties, Roster

In [ ]:
data_path = Path.cwd() / "data"
# roster_path = data_path / "202201_ZIB_Jan_anon.xlsx"
roster_path = data_path / "private" / "202201_ZIB_Jan.xlsx"

day_pattern = re.compile(r"MO|DI|MI|DO|FR|SA|SO")
date_pattern = re.compile(r"\d{2}")

def daycol(col):
    return day_pattern.search(col)

# TODO: Should use daycols for variable length months
df = pd.read_excel(roster_path, skiprows=0, header=1, index_col=0)
roster_df = (
    df.dropna().rename_axis("Name").loc[:, [col for col in df.columns if daycol(col)]]
)
roster_df

,SA 01,SO 02,MO 03,DI 04,MI 05,DO 06,FR 07,SA 08,SO 09,MO 10,...,SA 22,SO 23,MO 24,DI 25,MI 26,DO 27,FR 28,SA 29,SO 30,MO 31
Name,,,,,,,,,,,,,,,,,,,,,
"Schneider, Niklas",(/),(/),F0,F0,F0,(/),F0,(/),(/),F0,...,(/),(/),F0,F0,F0,F0,F0,(/),(/),F0
"Lärmer, Jürgen",(/),(/),F0,F0,F0,(/),F0,(/),(/),F0,...,(/),(/),F0,F0,F0,F0,F0,(/),(/),F0
"Müller, Georg",(/),(/),F1,F1,F1,(/),N1,N2,N2,(/),...,U,U,N1,N1,(/),(/),F1,F2,F2,S1
"Hartmann, Volker",(/),(/),(/),S1,(/),(/),S1,(/),(/),(/),...,U,U,U,U,U,U,U,U,U,N1
"Rusche, Katharina",(/),N2,N1,(/),(/),(/),U,(/),(/),S1,...,(/),(/),S1,ZA,ZA,S1,N1,N2,N2,(/)
"Westrick, Oliver",(/),(/),U,U,U,(/),3g,(/),(/),N1,...,(/),(/),3g,3g,3g,3g,3g,(/),(/),3g
"Schott, Johannes",(/),(/),S1,N1,N1,N2,(/),(/),(/),(/),...,F2,F2,(/),S1,N1,N1,(/),(/),(/),F1
"Ehlert, Nina",F2,F2,(/),(/),S1,(/),(/),(/),(/),(/),...,(/),(/),ZA,(/),(/),(/),S1,(/),(/),ZA
"Weiß, Theresa",N2,(/),(/),(/),(/),(/),F1,F2,F2,(/),...,N2,N2,(/),(/),F1,F1,F1,U,U,U


In [ ]:
_abbr2shiftproperties = {
    "(/)": None,
    "2M": None,
    "3g": None,
    "F0": ShiftProperties(
        name="Früh0", starting_hour=timedelta(hours=8), duration=timedelta(hours=10)
    ),
    "F1": ShiftProperties(
        name="Früh1", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "F2": ShiftProperties(
        name="Früh2", starting_hour=timedelta(hours=8), duration=timedelta(hours=12)
    ),
    "FB": ShiftProperties(
        name="Fortbildung",
        starting_hour=timedelta(hours=8),
        duration=timedelta(hours=8),
    ),
    "N1": ShiftProperties(
        name="Nacht1", starting_hour=timedelta(hours=22), duration=timedelta(hours=10)
    ),
    "N2": ShiftProperties(
        name="Nacht2", starting_hour=timedelta(hours=20), duration=timedelta(hours=12)
    ),
    "S1": ShiftProperties(
        name="Spät1", starting_hour=timedelta(hours=15), duration=timedelta(hours=8)
    ),
    "U": ShiftProperties(
        name="Urlaub", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "ZA": ShiftProperties(
        name="Zeitausgleich",
        starting_hour=timedelta(hours=8),
        duration=timedelta(hours=8),
    ),
}


In [ ]:
for name in roster_df.index:
    r = Roster.from_dict(roster_df.loc[name].to_dict(), mapper=_abbr2shiftproperties)
    filename = f"{name.replace(',', '_').replace(' ', '')}.ics"
    with (data_path / "output" / filename).open('w') as f:
        f.write(str(r.to_ics()))
    